<a href="https://colab.research.google.com/github/EltonWolf/GoogleEarthPythonBootcampAPI-/blob/main/Counties_in_the_US.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import ee

In [2]:
ee.Authenticate()
ee.Initialize()

To authorize access needed by Earth Engine, open the following URL in a web browser and follow the instructions. If the web browser does not start automatically, please manually browse the URL below.

    https://code.earthengine.google.com/client-auth?scopes=https%3A//www.googleapis.com/auth/earthengine%20https%3A//www.googleapis.com/auth/devstorage.full_control&request_id=DeFeTP_vc9tgEbVMA-qV63MC1LmiNL--sRky-1ARYZE&tc=vHeUAiWx4xAwngqFHGKnKbCHpQN5_2IZNhAHc6nh22A&cc=N3iDlsJHAIUxRPqL0zslKbcf9vbwciclxeFLYI5_8o0

The authorization workflow will generate a code, which you should paste in the box below.
Enter verification code: 4/1AfJohXkpDMx152qAtoUvUd_0ROl7EFYrK6cP5SIgL-s6WuFmblJi8oYpL70

Successfully saved authorization token.


In [36]:
dataset = ee.FeatureCollection("TIGER/2018/Counties")
Visual_Parameters = {
    'palette': ['purple', 'green', 'blue', 'yellow', 'orange', 'red'],
    'min': 0,
    'max': 50,
    'opacity': 0.8,
}

In [37]:
dataset = dataset.map(lambda f: f.set("STATEFP", ee.Number.parse(f.get("STATEFP"))))

image = ee.Image().float().paint(dataset, "STATEFP")

county_outlines = ee.Image().float().paint(**{
    'featureCollection': dataset,
    'color': 'black',
    'width': 1
})

In [38]:
import folium

In [39]:
# Define a method for displaying Earth Engine image tiles to folium map.
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
    tiles = map_id_dict['tile_fetcher'].url_format,
    attr = 'Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
    name = name,
    overlay = True,
    control = True
  ).add_to(self)

# Add EE drawing method to folium.
folium.Map.add_ee_layer = add_ee_layer

# Create a folium map object.
my_map = folium.Map(location=[37.649, -99.844], zoom_start = 3.5)

# Add the layer to the map object.
my_map.add_ee_layer(image, Visual_Parameters, "TIGER/2018/Counties")
my_map.add_ee_layer(county_outlines, {}, 'County Outlines')

# Add a layer control panel to the map.
my_map.add_child(folium.LayerControl())

# Display the map.
display(my_map)